In [ ]:
import time
import json
import re
import traceback

from selenium import webdriver
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    TimeoutException,
    ElementNotInteractableException,
)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service


# ---------------------------- Helper Functions ----------------------------
def wait_for_element(driver, xpath, duration=30):
    """
    Wait until an element matching xpath is present in the DOM.
    Returns the WebElement if found within 'duration' seconds.
    """
    return WebDriverWait(driver, duration).until(
        EC.presence_of_element_located((By.XPATH, xpath))
    )


def wait_for_element_clickable(driver, xpath, duration=30):
    """
    Wait until an element matching xpath is clickable.
    Returns the WebElement if found within 'duration' seconds.
    """
    return WebDriverWait(driver, duration).until(
        EC.element_to_be_clickable((By.XPATH, xpath))
    )


def dismiss_alert(driver):
    """
    If an alert is present, accept it. Otherwise, do nothing.
    """
    try:
        alert = driver.switch_to.alert
        alert.accept()
        print("Alert was present and accepted.")
    except NoAlertPresentException:
        pass


def is_delete_button_present(driver):
    """
    Check for the presence of the 'delete' button on the video upload interface.
    Used as a signal that the video has finished uploading.
    """
    try:
        driver.find_element(By.XPATH, '//*[contains(text(), "删除")]')
        print("Delete button found, upload likely complete.")
        return True
    except NoSuchElementException:
        return False


def clean_title(title):
    """
    Remove disallowed characters. Keep letters, digits, Chinese characters,
    and a few special symbols: 「」"“”:+?%° plus space.
    Commas become spaces.
    """
    title = title.replace(',', ' ')
    allowed_chars_regex = r'[a-zA-Z0-9\u4e00-\u9fff「」"“”:\+\?%° ]'
    return ''.join(re.findall(allowed_chars_regex, title))


def clear_and_type(element, text):
    """
    Clear a text field and type the given 'text' with a short pause.
    """
    element.clear()
    time.sleep(1)
    element.send_keys(text)


# ---------------------------- Main Publishing Function ----------------------------
def publish_shipinhao(driver, video_path, thumbnail_path, metadata, test=False):
    """
    Streamlined function to automate the publishing steps on 视频号.
    Assumes the user is already logged in on 'driver'.
    """
    try:
        print("Navigating to create-post page...")
        driver.get("https://channels.weixin.qq.com/post/create")
        dismiss_alert(driver)
        time.sleep(10)

        # Wait for "发表动态" label to appear, meaning page is loaded
        wait_for_element(driver, "//span[contains(text(), '发表动态')]", 30)
        dismiss_alert(driver)
        time.sleep(3)

        # ------------------ Step 1: Upload video ------------------
        print("Uploading video...")
        video_upload_input = wait_for_element(driver, '//input[@type="file"]', 30)
        video_upload_input.send_keys(video_path)

        start_time = time.time()
        timeout = 3600  # 1 hour
        while not is_delete_button_present(driver):
            if time.time() - start_time > timeout:
                raise TimeoutException("Video upload took too long or delete button not found.")
            print("Waiting for the video to finish uploading...")
            dismiss_alert(driver)
            time.sleep(5)

        # Give a few more seconds after noticing the delete button
        time.sleep(10)
        print("Video upload complete or delete button detected.")

        # ------------------ Step 2: Upload/confirm cover ------------------
        try:
            print("Uploading cover...")
            # Click "更换封面"
            change_cover_btn = wait_for_element_clickable(
                driver, '//*[contains(text(), "更换封面")]', 15
            )
            change_cover_btn.click()
            time.sleep(2)

            # Click "上传封面"
            upload_cover_btn = wait_for_element_clickable(
                driver, '//*[contains(text(), "上传封面")]', 15
            )
            upload_cover_btn.click()
            time.sleep(2)

            # Input file path for cover image
            file_input_xpath = '//input[@type="file" and @accept="image/jpeg,image/jpg,image/png"]'
            file_input = wait_for_element(driver, file_input_xpath, 15)
            file_input.send_keys(thumbnail_path)
            time.sleep(2)

            # Confirm upload
            confirm_btn_xpath = '//*[contains(@class, "finder-dialog-footer")]//*[contains(text(), "确认")]'
            confirm_btn = wait_for_element_clickable(
                driver, confirm_btn_xpath, 15
            )
            confirm_btn.click()
            time.sleep(2)
            print("Cover uploaded and confirmed.")
        except Exception as cover_ex:
            print("Cover upload error:", cover_ex)
            # Attempt to close any open dialog
            try:
                close_btn = wait_for_element_clickable(
                    driver,
                    '//button[contains(@class, "weui-desktop-dialog__close-btn")]',
                    5
                )
                close_btn.click()
                print("Closed cover upload dialog.")
            except:
                pass

        # ------------------ Step 3: Set description ------------------
        print("Setting description with tags...")
        desc_box = wait_for_element(driver, '//*[@data-placeholder="添加描述"]', 30)
        video_description = metadata.get("long_description", "")
        tags = metadata.get("tags", [])
        all_tags = " ".join("#" + tag for tag in tags)
        video_description_with_tags = f"{video_description} {all_tags}".strip()
        clear_and_type(desc_box, video_description_with_tags)
        time.sleep(3)

        # ------------------ Step 4: Pick collection ------------------
        print("Selecting collection (e.g., 简单生活)...")
        collection_dropdown = wait_for_element_clickable(
            driver, 
            "//div[@class='post-album-display-wrap']//div[contains(text(), '选择合集')]",
            30
        )
        collection_dropdown.click()
        time.sleep(3)

        # Example: Select "简单生活" from the dropdown
        simple_life = wait_for_element_clickable(
            driver,
            "//div[@class='common-option-list-wrap option-list-wrap']"
            "//div[@class='item']//div[text()='简单生活']",
            30
        )
        simple_life.click()
        time.sleep(3)

        # ------------------ Step 5: Set short title ------------------
        print("Setting short title...")
        raw_title = metadata.get("title", "Untitled")
        brief_desc = metadata.get("brief_description", "")
        # Fallback if title length not in [6..16]
        if not (6 <= len(raw_title) <= 16):
            raw_title = brief_desc[:16] if len(brief_desc) >= 6 else "Video"
        short_title_input = wait_for_element(
            driver,
            '//input[@placeholder="概括视频主要内容，字数建议6-16个字符"]',
            30
        )
        short_title = clean_title(raw_title[:16])
        clear_and_type(short_title_input, short_title)
        time.sleep(3)

#         # ------------------ Step 6: Declare original & pick category ------------------
#         print("Declaring original content...")
#         original_checkbox = driver.find_element(
#             By.XPATH,
#             '//input[@class="ant-checkbox-input" and @type="checkbox"]'
#         )
#         if not original_checkbox.is_selected():
#             original_checkbox.click()
#             time.sleep(3)

#         return
            
#         # Expand category dropdown
#         dropdown_label = driver.find_element(
#             By.XPATH, '//div[contains(@class,"weui-desktop-form__dropdown")]//dl'
#         )
#         dropdown_label.click()
#         time.sleep(3)

#         # Select "生活"
#         life_option = driver.find_element(
#             By.XPATH, '//span[@class="weui-desktop-dropdown__list-ele__text" and text()="生活"]'
#         )
#         life_option.click()
#         time.sleep(3)

#         # Check the agreement
#         agreement_checkbox = driver.find_element(
#             By.XPATH,
#             '//div[@class="original-proto-wrapper"]//input[@type="checkbox"]'
#         )
#         if not agreement_checkbox.is_selected():
#             agreement_checkbox.click()
#             time.sleep(3)

#         # Click the "声明原创" button
#         declare_btn = wait_for_element_clickable(
#             driver, 
#             '//div[@class="weui-desktop-dialog__ft"]//button[contains(text(), "声明原创")]',
#             30
#         )
#         declare_btn.click()
#         time.sleep(3)

#         # ------------------ Step 8: "声明原创" dialog (new approach)  ------------------
#         print("Handling the new original declaration dialog...")

#         # 1) Find the 'ant-checkbox-input' for the agreement
#         #    Use JS-based click to avoid intercept
#         try:
#             agreement_input = wait_for_element(driver, '//input[@class="ant-checkbox-input"]', 10)
#             if not agreement_input.is_selected():
#                 driver.execute_script("arguments[0].click();", agreement_input)
#                 time.sleep(2)
#         except Exception as exc:
#             print("Agreement checkbox issue:", exc)

#         # 2) Click the "声明原创" button
#         try:
#             declare_btn = wait_for_element_clickable(
#                 driver,
#                 '//button[@type="button" and contains(@class, "weui-desktop-btn_primary") and contains(text(), "声明原创")]',
#                 30
#             )
#             driver.execute_script("arguments[0].click();", declare_btn)
#             time.sleep(3)
#             print("Original declaration submitted.")
#         except Exception as exc:
#             print("Could not click the '声明原创' button:", exc)

        # ------------------ Step 6: Declare original content ------------------
        print("Declaring original content...")
        driver.execute_script(
            """
            // Step 1: Click the first checkbox to open the dialog
            let declareOriginalCheckbox = document.querySelector('.declare-original-checkbox input.ant-checkbox-input');

            if (declareOriginalCheckbox) {
                // Simulate a click event on the first checkbox
                let event = new MouseEvent('click', {
                    bubbles: true,
                    cancelable: true,
                    view: window,
                });
                declareOriginalCheckbox.dispatchEvent(event);
                console.log('First checkbox clicked to open dialog.');

                setTimeout(() => {
                    // Step 2: Find the dialog-specific checkbox
                    let dialogCheckbox = document.querySelector('.original-proto-wrapper input.ant-checkbox-input');
                    if (dialogCheckbox) {
                        dialogCheckbox.checked = true;
                        let changeEvent = new Event('change', { bubbles: true, cancelable: true });
                        dialogCheckbox.dispatchEvent(changeEvent);
                        console.log('Dialog checkbox programmatically checked.');

                        setTimeout(() => {
                            // Step 3: Find and click the "声明原创" button by text
                            let buttons = document.querySelectorAll('button');
                            let declareButton = Array.from(buttons).find(
                                btn => btn.textContent.trim() === '声明原创'
                            );
                            if (declareButton) {
                                let buttonEvent = new MouseEvent('click', {
                                    bubbles: true,
                                    cancelable: true,
                                    view: window,
                                });
                                declareButton.dispatchEvent(buttonEvent);
                                console.log('"声明原创" button clicked!');

                                setTimeout(() => {
                                    // Step 4: Close the dialog
                                    let closeButton = document.querySelector('.weui-desktop-icon-btn.weui-desktop-dialog__close-btn');
                                    if (closeButton) {
                                        let closeEvent = new MouseEvent('click', {
                                            bubbles: true,
                                            cancelable: true,
                                            view: window,
                                        });
                                        closeButton.dispatchEvent(closeEvent);
                                        console.log('Dialog closed!');
                                    } else {
                                        console.log('Close button not found!');
                                    }
                                }, 500);
                            } else {
                                console.log('"声明原创" button not found!');
                            }
                        }, 500);
                    } else {
                        console.log('Dialog checkbox not found!');
                    }
                }, 500);
            } else {
                console.log('First checkbox not found!');
            }
            """
        )
        time.sleep(5)

        # ------------------ Final Step: Publish ------------------
        if test:
            user_input = input("Type 'yes' to publish now: ").strip().lower()
        else:
            user_input = "yes"

        if user_input == "yes":
            print("Publishing now...")
            publish_btn = wait_for_element_clickable(driver, '//*[text()="发表"]', 30)
            publish_btn.click()
            time.sleep(10)
            print("Publish submitted.")
        else:
            print("User canceled publish.")

        print("Publishing workflow complete.")

    except Exception as e:
        print("Error during publishing:", e)
        traceback.print_exc()
        
chromedriver_path = '/usr/lib/chromium-browser/chromedriver'


        # Function to create a new WebDriver instance
def create_new_driver(port=5006):
    options = webdriver.ChromeOptions()
    options.add_experimental_option("debuggerAddress", f"127.0.0.1:{str(port)}")
    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# ---------------------------- Example Jupyter Usage ----------------------------
# 
# In a Jupyter cell:
#
# 1) Set up your WebDriver (Chrome or otherwise). Example:
#
# from selenium import webdriver
# options = webdriver.ChromeOptions()
# options.add_experimental_option("debuggerAddress", "127.0.0.1:5006")
# driver = webdriver.Chrome(options=options)
driver = create_new_driver(port=5006)
#

# 2) Provide paths to your data files. Adjust paths if needed.
video_path = "/home/lachlan/Projects/auto-publish/transcription_data/IMG_5197_2024_09_22_21_01_17_COMPLETED/IMG_5197_2024_09_22_21_01_17_COMPLETED_highlighted.mp4"
thumbnail_path = "/home/lachlan/Projects/auto-publish/transcription_data/IMG_5197_2024_09_22_21_01_17_COMPLETED/IMG_5197_2024_09_22_21_01_17_COMPLETED_cover.jpg"
metadata_path = "/home/lachlan/Projects/auto-publish/transcription_data/IMG_5197_2024_09_22_21_01_17_COMPLETED/IMG_5197_2024_09_22_21_01_17_COMPLETED_metadata.json"
#
# 3) Load metadata
with open(metadata_path, "r") as f:
    metadata = json.load(f)
#

# 4) Call publish_shipinhao with test=True so you have a chance to confirm or cancel
publish_shipinhao(driver, video_path, thumbnail_path, metadata, test=True)
#
# Note: You can break execution anywhere to inspect the DOM or debug.



Navigating to create-post page...
Alert was present and accepted.
Uploading video...
Waiting for the video to finish uploading...
Delete button found, upload likely complete.
Video upload complete or delete button detected.
Uploading cover...
Cover uploaded and confirmed.
Setting description with tags...
Selecting collection (e.g., 简单生活)...
Setting short title...
Declaring original content...
